## `shape` and `strides`

In [17]:
import numpy as np
a = np.arange(8, dtype=np.uint8)
a

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint8)

In [18]:
a.shape, a.strides

((8,), (1,))

In [19]:
b = a.reshape(2, 4)
b

array([[0, 1, 2, 3],
       [4, 5, 6, 7]], dtype=uint8)

In [20]:
b.shape, b.strides

((2, 4), (4, 1))

NumPy array is just a memory block with extra information how to interpret its contents. Since memory has only linear address space, NumPy arrays need extra information how to lay out this block into multiple dimensions. This is done by means of `shape`  and `strides` attributes:

![Shape and strides](strides.svg)


Note that strides are in bytes:

In [21]:
a_long = np.arange(8, dtype=np.int32).reshape(2,4)
a_long.strides

(16, 4)

To obtain number of bytes taken by a single element use `itemsize` property:

In [22]:
a_long.itemsize

4

## Exercise: Transpose

Create 3x4 random array. Compare the `shape` and `strides` properties of `x` and `x.T`. How can you
explain the new strides?

## Exercise: Broadcasting revisited

Explain how broadcasting works internally using the example below. What will be the `shape` and `strides` of `x` and `y` after broadcasting. Test it using `np.broadcast_arrays` in the following example and look at `strides` and `shape` properties of both arrays.

```
x = np.random.rand(5, 10)
y = np.random.rand(10)
z = x + y

xb, yb = np.broadcast_arrays(x, y)
```

## Manipulating strides

We my use 
`as_strided` function from NumPy library module  to manipulate the `shape` and `strides` properties (**Warning**: `as_strided` does not check if you remain in the memory bounds of the original array, so use with care!)

For example, we may define overlapping strides:

![Overlapping strides](strides2.svg)

In [23]:
a = np.arange(8, dtype=np.uint8)
a2 = np.lib.stride_tricks.as_strided(a, strides=(2, 1), shape=(3,4))
a2

array([[0, 1, 2, 3],
       [2, 3, 4, 5],
       [4, 5, 6, 7]], dtype=uint8)

Note that in this example some values appear twice, but they do not consume extra memory -- both arrays share the same memory block:

In [24]:
a[2] = 100
a

array([  0,   1, 100,   3,   4,   5,   6,   7], dtype=uint8)

In [25]:
a2

array([[  0,   1, 100,   3],
       [100,   3,   4,   5],
       [  4,   5,   6,   7]], dtype=uint8)

## Exercise: Sliding window

Use `as_strided` to produce a sliding-window view of a 1D array.

```python
def sliding_window(arr, size=2):
    """Produce an array of sliding window views of `arr`
    
    Parameters
    ----------
    arr : 1D array, shape (N,)
        The input array.
    size : int, optional
        The size of the sliding window.
        
    Returns
    -------
    arr_slide : 2D array, shape (N - size - 1, size)
        The sliding windows of size `size` of `arr`.
        
    Examples
    --------
    >>> a = np.array([0, 1, 2, 3])
    >>> sliding_window(a, 2)
    array([[0, 1],
           [1, 2],
           [2, 3]])
    """
    return arr  # fix this
```

## Fortran and C-ordering

In [26]:
c_order = np.arange(6).reshape(2, 3)
c_order

array([[0, 1, 2],
       [3, 4, 5]])

In [27]:
c_order.strides

(24, 8)

Let's see how the array is stored in memory.

In [28]:
np.ravel(c_order, order='A')

array([0, 1, 2, 3, 4, 5])

In C, the last index changes most rapidly as one moves through the array as stored in memory. 

In [29]:
a[0]

0

In [30]:
fortran_order = np.array(c_order, order='F')
fortran_order

array([[0, 1, 2],
       [3, 4, 5]])

In [31]:
fortran_order.strides

(8, 16)

In [32]:
np.ravel(fortran_order, order='A')

array([0, 3, 1, 4, 2, 5])

In Fortran the first index is the most rapidly varying index when moving through the elements of a two dimensional array as it is stored in memory.

### Exercise: Fastest changing index 
 
Compare the time of summing over rows and columns of an array `A = np.random.rand(3000, 3000)`. How would you explain the differences? (*Hint*: To measure evaluation time you can use `%timeit` of ipython)

## Further reading

* Internal-memory-layout-of-an-ndarray, http://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html#internal-memory-layout-of-an-ndarray
* Multidimensional Array Indexing Order Issues, http://docs.scipy.org/doc/numpy/reference/internals.html#multidimensional-array-indexing-order-issues